# Data Leakage Test

**Cel:** Sprawdzić czy model ma data leakage

**Test:**
1. Model Z data leakage (wszystkie cechy)
2. Model BEZ data leakage (tylko bezpieczne cechy)
3. Porównanie wyników

In [1]:
# Imports
import sys
sys.path.append('../')

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from config.spark_config import SparkConfig
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
import pandas as pd

In [2]:
# Start Spark
spark = SparkConfig.get_spark_session("DataLeakageTest")
print("Spark ready!")

✓ Windows: HADOOP_HOME set to C:\Users\usate\PycharmProjects\BlackFriday\hadoop
  Note: If you encounter permission errors, download winutils.exe
  from https://github.com/steveloughran/winutils and place in hadoop/bin/
Spark Session Created: DataLeakageTest
Spark Version: 3.5.3
Master: local[*]
App Name: DataLeakageTest
Spark ready!


In [3]:
# Load data
print("Loading data...")
df_features = spark.read.format("delta").load("../data/processed/delta/features")
print(f"Loaded: {df_features.count():,} rows")

# Split train/test
train_data, test_data = df_features.randomSplit([0.8, 0.2], seed=42)
train_data.cache()
test_data.cache()

print(f"Train: {train_data.count():,} rows")
print(f"Test: {test_data.count():,} rows")

Loading data...
Loaded: 550,068 rows
Train: 440,174 rows
Test: 109,894 rows


## Test 1: Model Z Data Leakage (Original)

In [4]:
print("="*60)
print("TEST 1: Model Z DATA LEAKAGE (oryginalne cechy)")
print("="*60)

ml_data_leakage = train_data.select("Purchase", "scaled_features").withColumnRenamed("scaled_features", "features")
test_data_leakage = test_data.select("Purchase", "scaled_features").withColumnRenamed("scaled_features", "features")

# Train
rf_leakage = RandomForestRegressor(
    featuresCol="features",
    labelCol="Purchase",
    predictionCol="prediction",
    numTrees=100,
    maxDepth=10,
    seed=42
)

print("Training model WITH data leakage...")
rf_model_leakage = rf_leakage.fit(ml_data_leakage)
print("Training complete!")

TEST 1: Model Z DATA LEAKAGE (oryginalne cechy)
Training model WITH data leakage...
Training complete!


In [5]:
# Evaluate
predictions_leakage = rf_model_leakage.transform(test_data_leakage)

evaluator_rmse = RegressionEvaluator(labelCol="Purchase", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="Purchase", predictionCol="prediction", metricName="mae")
evaluator_r2 = RegressionEvaluator(labelCol="Purchase", predictionCol="prediction", metricName="r2")

rmse_leakage = evaluator_rmse.evaluate(predictions_leakage)
mae_leakage = evaluator_mae.evaluate(predictions_leakage)
r2_leakage = evaluator_r2.evaluate(predictions_leakage)

print("\nRESULTS WITH DATA LEAKAGE:")
print(f"  RMSE: ${rmse_leakage:,.2f}")
print(f"  MAE:  ${mae_leakage:,.2f}")
print(f"  R²:   {r2_leakage:.4f}")


RESULTS WITH DATA LEAKAGE:
  RMSE: $244.82
  MAE:  $162.84
  R²:   0.9976


## Test 2: Model BEZ Data Leakage (Safe Features Only)

In [6]:
print("="*60)
print("TEST 2: Model BEZ DATA LEAKAGE (bezpieczne cechy)")
print("="*60)

# Bezpieczne cechy (NIE zawierają Purchase w obliczeniach)
safe_features = [
    # Demographics
    "Occupation", "Marital_Status",
    
    # Product categories
    "Product_Category_1", "Product_Category_2", "Product_Category_3",
    
    # User features (BEZ Purchase-based)
    "user_purchase_count",          # liczba transakcji - OK
    "user_unique_products",         # liczba unikalnych produktów - OK
    "user_unique_categories",       # liczba unikalnych kategorii - OK
    
    # Product features (TYLKO liczby transakcji, NIE średnie ceny!)
    "product_purchase_count",       # liczba zakupów produktu - OK
    "product_unique_users",         # liczba unikalnych użytkowników - OK (poprawione z product_unique_customers)
    "product_popularity_score",     # ranking - OK
    
    # Category features
    "category_purchase_count",      # liczba transakcji w kategorii - OK
    
    # RFM - TYLKO frequency (monetary usunięty!)
    "rfm_frequency_score",          # OK - oparty na user_purchase_count
    
    # Interaction features - TYLKO te bez Purchase
    "is_high_value_customer"        # OK - oparty na rfm_frequency_score
]

print(f"\nSafe features count: {len(safe_features)}")
print("\nFeatures used:")
for f in safe_features:
    print(f"  ✓ {f}")

TEST 2: Model BEZ DATA LEAKAGE (bezpieczne cechy)

Safe features count: 14

Features used:
  ✓ Occupation
  ✓ Marital_Status
  ✓ Product_Category_1
  ✓ Product_Category_2
  ✓ Product_Category_3
  ✓ user_purchase_count
  ✓ user_unique_products
  ✓ user_unique_categories
  ✓ product_purchase_count
  ✓ product_unique_users
  ✓ product_popularity_score
  ✓ category_purchase_count
  ✓ rfm_frequency_score
  ✓ is_high_value_customer


In [7]:
# Cechy USUNIĘTE (z data leakage):
removed_features = [
    "user_total_spent",              # suma Purchase ❌
    "user_avg_purchase",             # średnia Purchase ❌
    "user_min_purchase",             # min Purchase ❌
    "user_max_purchase",             # max Purchase ❌
    "user_std_purchase",             # std Purchase ❌
    "product_avg_price",             # średnia z Purchase ❌
    "product_total_revenue",         # suma Purchase ❌
    "category_avg_price",            # średnia z Purchase ❌
    "category_total_revenue",        # suma Purchase ❌
    "rfm_monetary_score",            # z user_total_spent ❌
    "rfm_score",                     # zawiera rfm_monetary_score ❌
    "purchase_vs_user_avg_ratio",   # Purchase / user_avg ❌
    "purchase_vs_product_avg_ratio", # Purchase / product_avg ❌
    "is_above_user_avg"             # porównanie z Purchase ❌
]

print("\n❌ Features REMOVED (data leakage):")
for f in removed_features:
    print(f"  ❌ {f}")


❌ Features REMOVED (data leakage):
  ❌ user_total_spent
  ❌ user_avg_purchase
  ❌ user_min_purchase
  ❌ user_max_purchase
  ❌ user_std_purchase
  ❌ product_avg_price
  ❌ product_total_revenue
  ❌ category_avg_price
  ❌ category_total_revenue
  ❌ rfm_monetary_score
  ❌ rfm_score
  ❌ purchase_vs_user_avg_ratio
  ❌ purchase_vs_product_avg_ratio
  ❌ is_above_user_avg


In [8]:
# Assemble features
assembler = VectorAssembler(
    inputCols=safe_features,
    outputCol="safe_features",
    handleInvalid="skip"
)

train_safe = assembler.transform(train_data).select("Purchase", col("safe_features").alias("features"))
test_safe = assembler.transform(test_data).select("Purchase", col("safe_features").alias("features"))

print(f"✓ Using {len(safe_features)} safe features")

✓ Using 14 safe features


In [9]:
# Train
rf_safe = RandomForestRegressor(
    featuresCol="features",
    labelCol="Purchase",
    predictionCol="prediction",
    numTrees=100,
    maxDepth=10,
    seed=42
)

print("Training model WITHOUT data leakage...")
rf_model_safe = rf_safe.fit(train_safe)
print("Training complete!")

Training model WITHOUT data leakage...
Training complete!


In [10]:
# Evaluate
predictions_safe = rf_model_safe.transform(test_safe)

rmse_safe = evaluator_rmse.evaluate(predictions_safe)
mae_safe = evaluator_mae.evaluate(predictions_safe)
r2_safe = evaluator_r2.evaluate(predictions_safe)

print("\nRESULTS WITHOUT DATA LEAKAGE:")
print(f"  RMSE: ${rmse_safe:,.2f}")
print(f"  MAE:  ${mae_safe:,.2f}")
print(f"  R²:   {r2_safe:.4f}")


RESULTS WITHOUT DATA LEAKAGE:
  RMSE: $2,793.09
  MAE:  $2,107.26
  R²:   0.6914


## Comparison & Diagnosis

In [11]:
print("="*80)
print("COMPARISON: WITH vs WITHOUT DATA LEAKAGE")
print("="*80)

comparison = pd.DataFrame({
    'Model': ['WITH Data Leakage', 'WITHOUT Data Leakage'],
    'RMSE': [rmse_leakage, rmse_safe],
    'MAE': [mae_leakage, mae_safe],
    'R²': [r2_leakage, r2_safe],
    'Features Used': [27, len(safe_features)]
})

print(comparison.to_string(index=False))

print("\n" + "="*80)
print("DIAGNOSIS")
print("="*80)

# Oblicz różnicę
rmse_diff = rmse_safe - rmse_leakage
r2_diff = r2_leakage - r2_safe

print(f"\n📊 Performance Drop (Without Leakage):")
print(f"  RMSE increased by: ${rmse_diff:,.2f} ({(rmse_diff/rmse_leakage)*100:.1f}%)")
print(f"  R² decreased by: {r2_diff:.4f} ({(r2_diff/r2_leakage)*100:.1f}%)")

if r2_leakage > 0.99 and r2_safe < 0.90:
    print("\n🚨 CONFIRMED: Severe data leakage detected!")
    print("   The original model had unrealistically high R² due to features")
    print("   that contained information about the target variable (Purchase).")
elif r2_diff > 0.05:
    print("\n⚠️  LIKELY: Data leakage detected")
    print("   Significant performance drop suggests some features leaked target info")
else:
    print("\n✅ No significant data leakage detected")

print("\n💡 RECOMMENDATION:")
if r2_leakage > 0.99:
    print("  1. REMOVE all Purchase-based features from feature engineering")
    print("  2. Recalculate user/product features excluding current transaction")
    print("  3. Retrain models with clean features")
    print("  4. Expected realistic R²: 0.70 - 0.90")
else:
    print("  Model performance looks reasonable")

print("\n" + "="*80)

COMPARISON: WITH vs WITHOUT DATA LEAKAGE
               Model        RMSE         MAE       R²  Features Used
   WITH Data Leakage  244.816524  162.844924 0.997629             27
WITHOUT Data Leakage 2793.085837 2107.260335 0.691359             14

DIAGNOSIS

📊 Performance Drop (Without Leakage):
  RMSE increased by: $2,548.27 (1040.9%)
  R² decreased by: 0.3063 (30.7%)

🚨 CONFIRMED: Severe data leakage detected!
   The original model had unrealistically high R² due to features
   that contained information about the target variable (Purchase).

💡 RECOMMENDATION:
  1. REMOVE all Purchase-based features from feature engineering
  2. Recalculate user/product features excluding current transaction
  3. Retrain models with clean features
  4. Expected realistic R²: 0.70 - 0.90

